In [1]:
import sys
from pathlib import Path
import pandas as pd

# Add project root to Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

In [2]:
import src.load as load

sensor_locations_raw = load.load_sensor_locations()
sensor_counts_raw = load.load_sensor_counts()

In [3]:
import src.clean as clean


sensor_locations = clean.clean_sensor_locations(sensor_locations_raw)
sensor_counts = clean.clean_sensor_counts(sensor_counts_raw)


In [4]:
import src.transform as transform

loc_counts = transform.get_loc_counts(sensor_locations, sensor_counts)
loc_counts

,location_id,sensor_description,sensor_name_x,installation_date,location_type,location,count_id,pedestriancount,sensor_name_y,sensing_date_time
0,1,Bourke Street Mall (North),Bou292_T,2009-03-24,Outdoor,b'\x01\x01\x00\x00\x00R\xcf\x06\x89\xe2\x1eb@\...,1720240410,217,Bou292_T,2024-04-10 07:00:00
1,1,Bourke Street Mall (North),Bou292_T,2009-03-24,Outdoor,b'\x01\x01\x00\x00\x00R\xcf\x06\x89\xe2\x1eb@\...,1020241016,41,Bou292_T,2024-10-16 00:00:00
2,1,Bourke Street Mall (North),Bou292_T,2009-03-24,Outdoor,b'\x01\x01\x00\x00\x00R\xcf\x06\x89\xe2\x1eb@\...,1520240614,24,Bou292_T,2024-06-14 05:00:00
3,1,Bourke Street Mall (North),Bou292_T,2009-03-24,Outdoor,b'\x01\x01\x00\x00\x00R\xcf\x06\x89\xe2\x1eb@\...,1120240914,69,Bou292_T,2024-09-14 01:00:00
4,1,Bourke Street Mall (North),Bou292_T,2009-03-24,Outdoor,b'\x01\x01\x00\x00\x00R\xcf\x06\x89\xe2\x1eb@\...,1920250315,742,Bou292_T,2025-03-15 09:00:00
...,...,...,...,...,...,...,...,...,...,...
1513664,184,124 Elizabeth Street,Eli124_T,2025-06-28,Outdoor,b'\x01\x01\x00\x00\x00]\xe3\x12\xcb\xd6\x1eb@\...,184320250819,32,Eli124_T,2025-08-19 03:00:00
1513665,184,124 Elizabeth Street,Eli124_T,2025-06-28,Outdoor,b'\x01\x01\x00\x00\x00]\xe3\x12\xcb\xd6\x1eb@\...,1841120250708,1378,Eli124_T,2025-07-08 11:00:00
1513666,184,124 Elizabeth Street,Eli124_T,2025-06-28,Outdoor,b'\x01\x01\x00\x00\x00]\xe3\x12\xcb\xd6\x1eb@\...,1841520251230,2774,Eli124_T,2025-12-30 15:00:00
1513667,184,124 Elizabeth Street,Eli124_T,2025-06-28,Outdoor,b'\x01\x01\x00\x00\x00]\xe3\x12\xcb\xd6\x1eb@\...,184120251214,208,Eli124_T,2025-12-14 01:00:00
